### Compare new tariffs (Partner and P4)

In [55]:
import numpy as np
import pandas as pd
import pandasql as ps
from pandas import Series, DataFrame
from os.path import join

load_folder = 'c:/Users/balob/Documents/GITLAB/RB_BD/DATA/'
file_p4_new_tariffs = 'P4_20190410.csv'
file_partner_new_tariffs = 'Partner_1_6_2019.csv'
file_allowed_list = 'DMI.dmi_allowed_list_export_Tue_May_21_2019.csv'

file_s1_to_db = 'df_s1_insert_267.csv'
file_s1_to_db_100 = 'df_s1_insert_267_100.csv'

In [47]:
# Из базы данных OCS выгружаем TADIG

import pandas as pd
import pyodbc 
#Connect to OCSDBREP1 (BSS)
cnxn = pyodbc.connect('DRIVER={SQL Server};SERVER=172.18.11.82;DATABASE=10028;UID=BSS;PWD=iKQVm40AZAmyRaw72LeY')

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
'''
df_tadig = pd.read_sql_query(sql_srt, cnxn)

sql_srt=\
'''
SELECT
MO_CALL_RATE MOC,
MT_CALL_RATE MTC,
MO_SMS_RATE SMS,
DATA_RATE DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=267
AND end_date IS NULL
'''

df_ocs_partner = pd.read_sql_query(sql_srt, cnxn)
print('Partner OCS cost:')
display(df_ocs_partner.head(3))

sql_srt=\
'''
SELECT
IIF(CURRENCY_ID=2,MO_CALL_RATE,MO_CALL_RATE*1.2) MOC,
IIF(CURRENCY_ID=2,MT_CALL_RATE,MT_CALL_RATE*1.2) MTC,
IIF(CURRENCY_ID=2,MO_SMS_RATE,MO_SMS_RATE*1.2) SMS,
IIF(CURRENCY_ID=2,DATA_RATE,DATA_RATE*1.2) DATA,
NETWORK_ID,SPONSOR_ID,is_discounted
FROM ROAMING_PLAN_RULES WHERE roaming_plan_id=268
AND end_date IS NULL
'''

df_ocs_p4 = pd.read_sql_query(sql_srt, cnxn)
print('\nP4 OCS cost:')
df_ocs_p4 = df_ocs_p4
display(df_ocs_p4.head(3))

cnxn.close()

Partner OCS cost:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted
0,0.990086,0.990086,0.249622,10.238187,59.0,1.0,False
1,2.189060,2.189060,0.450132,14.333252,95.0,1.0,False
2,0.660066,0.000000,0.299730,9.997989,108.0,1.0,False



P4 OCS cost:


,MOC,MTC,SMS,DATA,NETWORK_ID,SPONSOR_ID,is_discounted
0,3.1824,0.0,0.3384,1.872,171.0,2.0,False
1,3.8160,0.0,0.9216,17.136,465.0,2.0,False
2,2.5920,0.0,0.7200,21.600,1181.0,2.0,False


In [48]:
# Загружаем новые тарифы Partner и P4 и Allowed list

df_partner = pd.read_csv(join(load_folder, file_partner_new_tariffs),sep=';')
print('Новые тарифы Partner:')
display(df_partner.head(3))

df_p4 = pd.read_csv(join(load_folder, file_p4_new_tariffs),sep=';')

df_p4.Data_Increment_KB.replace({'1 kB':1, '10 kB':10, '1kB':1}, inplace=True)
df_p4.Voice_Increment_SEC.replace({'30/1':30}, inplace=True)
df_p4.MTC.fillna(0, inplace=True)

print('Новые тарифы P4:')
display(df_p4.head(3))

df_al=pd.read_csv(join(load_folder, file_allowed_list),usecols=['Sponsor','PLMN code'])
df_al.rename({'PLMN code':'TADIG_CODE_AL','Sponsor':'SPONSOR'},inplace=True,axis=1)
df_al = df_al[df_al.SPONSOR.isin(['Partner_Combined','P4_Combined'])]
df_al = df_al.replace({'Partner_Combined':'S1','P4_Combined':'S2'})
df_al = pd.merge(df_al, df_tadig[['TADIG_CODE','NETWORK_ID']], how='left',left_on='TADIG_CODE_AL',right_on='TADIG_CODE')
df_al.drop('TADIG_CODE',inplace=True,axis=1)

Новые тарифы Partner:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB
0,ALBVF,27602,Vodafone AL,Albania,VF,0.03,0,0.0062,0.003,1,1
1,CZECM,23003,Vodafone CZ,Czech Republic,VF,0.03,0,0.0062,0.003,1,1
2,DEUD2,26202,Vodafone DE,Germany,VF,0.03,0,0.0062,0.003,1,1


Новые тарифы P4:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE
0,Albania,TELEKOM ALBANIA SH.A,ALBAM,0.449,0.0,0.052,0.040,1,1,yes,yes
1,Albania,ALBtelecom sh.a.,ALBEM,0.390,0.0,0.040,0.039,60,1,yes,yes
2,Anguilla,Cable and Wireless (Anguilla) Limited,AIACW,0.800,0.0,0.080,0.080,1,1,yes,no


In [49]:
#Объединяем тарифы с TADIG и Allowed list

df_partner_net_id = pd.merge(df_partner,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах Partner с добавлением network_id:')
display(df_partner_net_id[df_partner_net_id.TADIG_CODE.isnull()])

df_p4_net_id = pd.merge(df_p4,df_tadig,left_on='TADIG',right_on='TADIG_CODE',how='left')
print('Неизвестные TADIG в тарифах P4 с добавлением network_id:')
display(df_p4_net_id[df_p4_net_id.TADIG_CODE.isnull()])

df_s1_s2_temp = pd.merge(df_partner_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','Voice_Increment_SEC','Data_Increment_KB']],
                              df_p4_net_id[['MOC','MTC','SMS','MB','NETWORK_ID','P4_CAMEL','P4_LTE',
                                  'Voice_Increment_SEC','Data_Increment_KB']],
                    how='outer',on=['NETWORK_ID'],suffixes=('_S1','_S2')).drop_duplicates()
#print('Тарифы Partner и P4:')
#display(df_s1_s2_temp.head())

df_s1_s2 = pd.merge(df_s1_s2_temp,df_tadig,how='left',on='NETWORK_ID')
#print('Тарифы Partner и P4 с TADIG:')
#display(df_s1_s2.head())

df_s1_s2_sponsor = pd.merge(df_s1_s2,df_al,how='outer',on='NETWORK_ID')
#print('Тарифы Partner и P4 с привязкой к Allowed list:')
#display(df_s1_s2_sponsor.head())

# Результирующий файл

#f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S1','MOC_S2','MTC_S1','MTC_S2','SMS_S1','SMS_S2','MB_S1','MB_S2','SPONSOR']
f_list = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE',
          'MOC_S1','MTC_S1','SMS_S1','MB_S1',
          'MOC_S2','MTC_S2','SMS_S2','MB_S2','SPONSOR','TADIG_CODE_AL','NETWORK_ID','P4_CAMEL','P4_LTE'
         ,'Voice_Increment_SEC_S1','Data_Increment_KB_S1','Voice_Increment_SEC_S2','Data_Increment_KB_S2']
df_s1_s2_sponsor = df_s1_s2_sponsor[f_list].drop_duplicates()
#df_s1_s2_sponsor.to_csv(join(load_folder,'Partner_P4_Compare_1_1_2019.csv'),index=False)
print('\nТарифы Partner и P4 с привязкой к Allowed list:')
df_s1_s2_sponsor.head(3)

Неизвестные TADIG в тарифах Partner с добавлением network_id:


,TADIG,MCCMNC,PARTNER,COUNTRY,Operator,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID


Неизвестные TADIG в тарифах P4 с добавлением network_id:


,COUNTRY,PARTNER,TADIG,MOC,MTC,SMS,MB,Voice_Increment_SEC,Data_Increment_KB,P4_CAMEL,P4_LTE,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
146,Mozambique,MOVITEL S.A,MOZVG,0.7,0.0,0.07,0.042,60,10,yes,no,NaN,NaN,NaN,NaN



Тарифы Partner и P4 с привязкой к Allowed list:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S1,MTC_S1,SMS_S1,MB_S1,MOC_S2,MTC_S2,SMS_S2,MB_S2,SPONSOR,TADIG_CODE_AL,NETWORK_ID,P4_CAMEL,P4_LTE,Voice_Increment_SEC_S1,Data_Increment_KB_S1,Voice_Increment_SEC_S2,Data_Increment_KB_S2
0,Albania,Vodafone,ALBVF,0.03,0.0,0.0062,0.003,NaN,NaN,NaN,NaN,S1,ALBVF,12.0,NaN,NaN,1.0,1.0,NaN,NaN
1,Czech Republic,Vodafone Czech Republic a s,CZECM,0.03,0.0,0.0062,0.003,0.035,0.0,0.013,0.0060,S2,CZECM,238.0,yes,yes,1.0,1.0,1,1.0
2,Germany,Vodafone GmbH,DEUD2,0.03,0.0,0.0062,0.003,0.045,0.0,0.013,0.0063,S2,DEUD2,338.0,yes,yes,1.0,1.0,1,1.0


In [50]:
# Проверка наличия различных тарифов для одного и того же оператора.
# Операторы определяются по данным OCS

print('Partner:')
df_temp = df_partner_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s1 = df_temp[df_temp.values > 1]
display(s1)

print('\nP4:')
df_temp = df_p4_net_id.groupby('NETWORK_ID')['MOC','MTC','SMS','MB'].nunique()
s2 = df_temp[df_temp.values > 1]
display(s2)
#df_s1_s2_sponsor[df_s1_s2_sponsor.NETWORK_ID.isin(s1.index.values)]

Partner:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,



P4:


,MOC,MTC,SMS,MB
NETWORK_ID,,,,
412.0,2,1,1,1


In [56]:
# Проверить изменения тарифов P4

df_temp = pd.merge(df_s1_s2_sponsor, df_ocs_p4, how='outer', on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG P4:")
cols_p4 = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2'
           ,'MOC','MTC','SMS','DATA','P4_LTE','P4_CAMEL','is_discounted','NETWORK_ID']
display(df_temp.loc[((df_temp.MB_S2.notnull()) & (df_temp.SPONSOR_ID.isnull()) & (df_temp.TADIG_CODE.notnull())), cols_p4])


# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG P4:")
s2 = df_temp[(df_temp.MB_S2.isnull()) & (df_temp.SPONSOR_ID.notnull()) & (df_temp.is_discounted == True)].NETWORK_ID.values
display(df_tadig[df_tadig.NETWORK_ID.isin(s2)])

# Отличия в тарифах P4

print("\nОтличия в тарифах P4:")

#df2=df.loc[((df.MB_S2.notnull()) | (df.DATA.notnull())) & (df.is_discounted == False), cols_p4].copy()
df2 = df_temp.loc[((df_temp.MB_S2.notnull()) | (df_temp.DATA.notnull())) & (df_temp.is_discounted.notnull()), cols_p4].copy()

df_diff_s2 = df2[(df2.MB_S2.notnull()) & (\
     ~np.isclose(df2['MOC_S2'], df2['MOC'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df2['MTC_S2'], df2['MTC'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df2['SMS_S2'], df2['SMS'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df2['MB_S2'], df2['DATA'], atol=0.00001, equal_nan=True)
)]

display(df_diff_s2.head(3))

Новые TADIG P4:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S2,MTC_S2,SMS_S2,MB_S2,MOC,MTC,SMS,DATA,P4_LTE,P4_CAMEL,is_discounted,NETWORK_ID
36,Bangladesh,Grameenphone Ltd,BGDGP,0.60,0.0,0.035,0.019,NaN,NaN,NaN,NaN,no,no,NaN,66.0
66,Congo Dem. Rep,Africell,CODAC,0.70,0.0,0.140,4.200,NaN,NaN,NaN,NaN,no,no,NaN,208.0
160,Satellite Networks,AeroMobile AS,NORAM,1.12,0.0,0.280,1.400,NaN,NaN,NaN,NaN,no,no,NaN,796.0
204,Myanmar (Burma),Telenor Myanmar Limited,MMRTN,0.60,0.0,0.035,0.019,NaN,NaN,NaN,NaN,no,no,NaN,633.0



Пропавшие TADIG P4:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
164,Croatia,Hrvatski Telekom d d,HRVCN,224.0
238,Germany,E Plus Mobilfunk GmbH,DEUE1,328.0



Отличия в тарифах P4:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC_S2,MTC_S2,SMS_S2,MB_S2,MOC,MTC,SMS,DATA,P4_LTE,P4_CAMEL,is_discounted,NETWORK_ID
1,Czech Republic,Vodafone Czech Republic a s,CZECM,0.035,0.0,0.013,0.0060,0.0504,0.0,0.01872,0.008640,yes,yes,True,238.0
2,Germany,Vodafone GmbH,DEUD2,0.045,0.0,0.013,0.0063,0.0648,0.0,0.01872,0.009072,yes,yes,True,338.0
4,Greece,Vodafone,GRCCT,0.035,0.0,0.013,0.0060,0.0504,0.0,0.01872,0.008640,yes,yes,True,354.0


In [84]:
# Проверить изменения тарифов Partner

df_temp=pd.merge(df_s1_s2_sponsor, df_ocs_partner, how='outer', on='NETWORK_ID')

# Проверить, есть ли новые PLMN

print("Новые TADIG Partner:")
cols_partner = ['COUNTRY_NAME','NETWORK_NAME','TADIG_CODE' ,'MOC','MOC_S1','MTC','MTC_S1','SMS','SMS_S1','DATA','MB_S1'
               ,'is_discounted','NETWORK_ID']
display(df_temp.loc[((df_temp.MB_S1.notnull()) & (df_temp.SPONSOR_ID.isnull())), cols_partner])

# Проверить, не закрылись ли какие-нибудь сети

print("\nПропавшие TADIG Partner:")
s1 = df_temp[(df_temp.MB_S1.isnull()) & (df_temp.SPONSOR_ID.notnull()) & (df_temp.is_discounted == True)].NETWORK_ID.values
display(df_tadig[df_tadig.NETWORK_ID.isin(s1)].head(3))

# Отличия в тарифах

print("\nОтличия в тарифах Partner:")

#df2=df.loc[((df.MB_S2.notnull()) | (df.DATA.notnull())) & (df.is_discounted == False), cols_p4].copy()
df1 = df_temp.loc[((df_temp.MB_S1.notnull()) | (df_temp.DATA.notnull())) & (df_temp.is_discounted.notnull()), cols_partner].copy()

df_diff_s1 = df1[(df1.MB_S1.notnull()) & (\
     ~np.isclose(df1['MOC_S1'], df1['MOC'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df1['MTC_S1'], df1['MTC'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df1['SMS_S1'], df1['SMS'], atol=0.00001, equal_nan=True) |\
     ~np.isclose(df1['MB_S1'], df1['DATA'], atol=0.00001, equal_nan=True)
)].copy()

display(df_diff_s1)

Новые TADIG Partner:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC,MOC_S1,MTC,MTC_S1,SMS,SMS_S1,DATA,MB_S1,is_discounted,NETWORK_ID



Пропавшие TADIG Partner:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,NETWORK_ID
13,Andorra,Andorra Telecom S A U,ANDMA,17.0
16,Anguilla,Cable and Wireless,AIACW,20.0
18,Antigua and Barbuda,C & W,ATGCW,23.0



Отличия в тарифах Partner:


,COUNTRY_NAME,NETWORK_NAME,TADIG_CODE,MOC,MOC_S1,MTC,MTC_S1,SMS,SMS_S1,DATA,MB_S1,is_discounted,NETWORK_ID
0,Albania,Vodafone,ALBVF,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,12.0
1,Czech Republic,Vodafone Czech Republic a s,CZECM,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,238.0
2,Germany,Vodafone GmbH,DEUD2,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,338.0
3,Ghana,Vodafone,GHAGT,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,343.0
4,Greece,Vodafone,GRCCT,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,354.0
5,Greece,Vodafone,GRCPF,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,354.0
6,Hungary,Vodafone Hungary Mobile Telecommunications Com...,HUNVR,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,400.0
7,Ireland,Vodafone Ireland Limited,IRLEC,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,463.0
8,Italy,Vodafone Italia S p A,ITAOM,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0014,True,485.0
9,Malta,Vodafone - MLTTL,MLTTL,0.0439,0.03,0.0,0.0,0.0062,0.0062,0.003,0.0030,True,597.0


In [85]:
# Подготовить файлы с различающимися тарифами для загрузки тарифов Partner в OCS

df_diff_s1['ROAMING_PLAN_RULE_ID'] = ''
df_diff_s1['ROAMING_PLAN_ID']='267'
df_diff_s1['SPONSOR_ID'] = 1
df_diff_s1['MT_SMS_RATE'] = 0
df_diff_s1['CURRENCY_ID'] = 1 #1:USD, 2:EUR
df_diff_s1['START_DATE'] = '2019-06-01 00:00:00'
df_diff_s1['END_DATE'] = 'NULL'

df_diff_s1_100 = df_diff_s1.copy()
df_diff_s1_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']] = df_diff_s1_100[['MOC_S1','MTC_S1','SMS_S1','MB_S1']]*2

df_diff_s1[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder,file_s1_to_db),index=False)

df_diff_s1_100[[#'TADIG_CODE',
'ROAMING_PLAN_RULE_ID',
'ROAMING_PLAN_ID',
'NETWORK_ID',
'SPONSOR_ID',
'MOC_S1',
'MTC_S1',
'SMS_S1',
'MT_SMS_RATE',
'MB_S1',
'CURRENCY_ID',
'START_DATE',
'END_DATE',
'is_discounted']].drop_duplicates().to_csv(join(load_folder,file_s1_to_db_100),index=False)

In [20]:
# Новые тарифы P4 для загрузки в RoamDb

file_partner_roamdb = 'p4_roamdb_result.csv'

print("Новые TADIG P4:")
cols_p4 = ['TADIG_CODE','MOC_S2','MTC_S2','SMS_S2','MB_S2'
           ,'P4_LTE','Voice_Increment_SEC','Data_Increment_KB']

df_rdb = df.loc[((df.MB_S2.notnull()) & (df.SPONSOR_ID.isnull())
                 & (df.TADIG_CODE.notnull())), cols_p4]

df_rdb['IsDiscounted (Yes/No)'] = 'Yes'
df_rdb.rename({'TADIG_CODE':'TADIG CODE','MOC_S2':'Mobile Origination','MTC_S2':'Mobile Termination','SMS_S2':'SMS','MB_S2':'DATA',
          'Voice_Increment_SEC':'Voice Increment','Data_Increment_KB':'Data Increment','P4_LTE':'Support4G(Yes/No)'},
         axis=1,inplace=True)

df_rdb['Support4G(Yes/No)'].replace({'yes':'Yes','no':'No'},inplace=True)

columns_ordered = ['TADIG CODE',
 'Mobile Origination',
 'Mobile Termination',
 'SMS',
 'DATA',
 'Voice Increment',
 'Data Increment',
 'IsDiscounted (Yes/No)',
 'Support4G(Yes/No)',]

df_rdb[columns_ordered].to_csv(join(load_folder,file_partner_roamdb),index=False)

df_rdb[columns_ordered]

Новые TADIG P4:


,TADIG CODE,Mobile Origination,Mobile Termination,SMS,DATA,Voice Increment,Data Increment,IsDiscounted (Yes/No),Support4G(Yes/No)
36,BGDGP,0.60,0.0,0.035,0.019,1,1.0,Yes,No
66,CODAC,0.70,0.0,0.140,4.200,1,1.0,Yes,No
160,NORAM,1.12,0.0,0.280,1.400,60,1.0,Yes,No
204,MMRTN,0.60,0.0,0.035,0.019,1,1.0,Yes,No
